<a href="https://colab.research.google.com/github/Vaishnavilande/ML-Projects-Machine-Learning-/blob/main/Linear%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing dependencies
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("/content/Medical_Price.csv")

In [3]:
df.shape

(1338, 7)

In [4]:
#missing value
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [5]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


#Data processing

In [6]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [8]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [10]:
#columns with object type values
df.select_dtypes('object').columns

Index(['sex', 'smoker', 'region'], dtype='object')

In [11]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{'sex': 2, 'smoker': 2, 'region': 4}

In [12]:
{column: list(df[column].unique()) for column in df.select_dtypes('object').columns}

{'sex': ['female', 'male'],
 'smoker': ['yes', 'no'],
 'region': ['southwest', 'southeast', 'northwest', 'northeast']}

In [13]:
#create function for gender encoding 

def encode_sex(x):
    if x.lower()[0] == 'f':   #if first letter of gender ie 0 is equal to f 
        return 0
    elif x.lower()[0] == 'm':
        return 1
    else:
        return 2

In [14]:
#apply the above function
df['sex'] = df['sex'].apply(encode_sex)

In [15]:
#create function for gender encoding 

def encode_smoker(x):
    if x.lower()[0] == 'y':   #if first letter of gender ie 0 is equal to f 
        return 0
    elif x.lower()[0] == 'n':
        return 1
    else:
        return 2

In [16]:
#apply the above function
df['smoker'] = df['smoker'].apply(encode_smoker)

In [17]:
nominal_features = [
    'region'
   
]

In [18]:
def onehot_encode(data, columns, prefixes):
    data = data.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix)
        data = pd.concat([data, dummies], axis=1)
        data = data.drop(column, axis=1)
    return data

In [19]:
nominal_prefixes = [
    'Re',
    
]

In [20]:
df = onehot_encode(
    df,
    columns=nominal_features,
    prefixes=nominal_prefixes
)

In [21]:
df

,age,sex,bmi,children,smoker,charges,Re_northeast,Re_northwest,Re_southeast,Re_southwest
0,19,0,27.900,0,0,16884.92400,0,0,0,1
1,18,1,33.770,1,1,1725.55230,0,0,1,0
2,28,1,33.000,3,1,4449.46200,0,0,1,0
3,33,1,22.705,0,1,21984.47061,0,1,0,0
4,32,1,28.880,0,1,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,1,10600.54830,0,1,0,0
1334,18,0,31.920,0,1,2205.98080,1,0,0,0
1335,18,0,36.850,0,1,1629.83350,0,0,1,0
1336,21,0,25.800,0,1,2007.94500,0,0,0,1


#Train test spliting

In [22]:
X = df.drop('smoker', axis = 1)
Y = df['smoker']

In [23]:
X.shape

(1338, 9)

In [24]:
Y.shape

(1338,)

In [26]:
x_train = X[:858] # this functions splits the x to x_train having first 1070 rows
x_test = X[859:] #this function splits the x to have x_test to have rows from 1070 onwards

y_train = Y[:858] #doing the same for y_train and Y_test
y_test = Y[859:]

y_train_reshape = y_train.values.reshape(-1,1) #reshape the training and testing dataset so that it becomes a dataframe from a series.
y_test_reshape = y_test.values.reshape(-1,1) 

#function for linear regresssion.

In [27]:
class Linear_Regression():

   def __init__( self, learning_rate, no_of_iterations ) :
          
        self.learning_rate = learning_rate
          
        self.no_of_iterations = no_of_iterations

    # fit function to train the model

   def fit( self, X, Y ) :
          
        # no_of_training_examples, no_of_features
          
        self.m, self.n = X.shape
          
        # initiating the weight and bias
          
        self.w = np.zeros( self.n )
          
        self.b = 0
          
        self.X = X
          
        self.Y = Y


        # implementing Gradient Descent for Optimization
                  
        for i in range( self.no_of_iterations ) :
              
            self.update_weights()
              
        
      
    # function to update weights in gradient descent
      
   def update_weights( self ) :
             
        Y_prediction = self.predict( self.X )
          
        # calculate gradients  
      
        dw = - ( 2 * ( self.X.T ).dot( self.Y - Y_prediction )  ) / self.m
       
        db = - 2 * np.sum( self.Y - Y_prediction ) / self.m 
          
        # updating the weights
      
        self.w = self.w - self.learning_rate * dw
      
        self.b = self.b - self.learning_rate * db
          
      
    # Line function for prediction:
      
   def predict( self, X ) :
      
        return X.dot( self.w ) + self.b

In [28]:
model = Linear_Regression(learning_rate = 0.02, no_of_iterations=1000)

In [30]:
model.fit(x_train, y_train)

<ipython-input-27-c13bb424616a>:46: RuntimeWarning: overflow encountered in double_scalars
  db = - 2 * np.sum( self.Y - Y_prediction ) / self.m
<ipython-input-27-c13bb424616a>:52: RuntimeWarning: invalid value encountered in double_scalars
  self.b = self.b - self.learning_rate * db


In [39]:
test_data_prediction = model.predict(x_test)


In [40]:
print(test_data_prediction)

859    NaN
860    NaN
861    NaN
862    NaN
863    NaN
        ..
1333   NaN
1334   NaN
1335   NaN
1336   NaN
1337   NaN
Length: 479, dtype: float64
